In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

weather_df = pd.read_csv('output_data/cities.csv').drop(['Unnamed: 0'],axis=1)
weather_df

,Cities,Latitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,ossora,59.2353,272.24,93,100,1.74
1,hilo,19.7297,292.34,83,90,1.54
2,clyde river,70.4692,268.15,100,40,10.29
3,georgetown,5.4112,300.22,89,20,1.54
4,hasaki,35.7333,281.52,49,20,6.69
...,...,...,...,...,...,...
554,tsabong,-26.0217,296.81,19,63,2.29
555,haines junction,60.7522,275.15,60,3,1.54
556,katsuura,35.1333,284.34,54,75,7.72
557,porto novo,6.4965,303.15,74,40,5.66
